In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [148]:
# Load the chessboard image
image = cv2.imread('image.png')

scale = 0.5
width = int(image.shape[1] * scale)
height = int(image.shape[0] * scale)

image = cv2.resize(image, (width, height))
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [149]:
blurred = cv2.GaussianBlur(gray, (5, 5), 1)

In [ ]:
# Use Hough Line Transform to detect lines in the image
edges = cv2.Canny(blurred, 60, 200, apertureSize=3)
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=10)
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
img_contours = image.copy()


# Create a copy of the original image to draw lines on
image_with_lines = image.copy()

for contour in contours:
    # Approximate the contour to a polygon
    epsilon = 0.15 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    # We're looking for quadrilaterals (4-sided polygons)
    if len(approx) == 4:
        # Optionally, filter by area to avoid small contours
        area = cv2.contourArea(contour)
        if 1000 < area < 10000:  # Filter for reasonable square sizes
            cv2.drawContours(img_contours, [approx], -1, (0, 255, 0), 2)

# Show the result
plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(img_contours, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
import torch
import cv2

# Load a pre-trained YOLOv5 model (fine-tuned for chess piece detection)
model = torch.hub.load('ultralytics/yolov5')  # 'best.pt' is your trained model

# Load the chessboard image
img = cv2.imread('chessboard_image.png')

# Perform inference on the image
results = model(img)

# Get the detected results (bounding boxes, labels, and confidences)
detections = results.xyxy[0]  # Bounding boxes and labels

# Display the image with bounding boxes
results.show()

# Loop through detected chess pieces
for det in detections:
    x1, y1, x2, y2, conf, cls = det  # Bounding box coordinates and label
    label = model.names[int(cls)]  # Convert class index to label name (e.g., 'rook', 'queen')
    
    print(f"Detected: {label} at [{x1}, {y1}, {x2}, {y2}] with confidence {conf}")


In [4]:
!pip install ultralytics


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\faree\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import ultralytics
import torch
from ultralytics import YOLO
import os
from PIL import Image
import cv2
from IPython.display import display
import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

ultralytics.checks()

Ultralytics 8.3.5  Python-3.11.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
Setup complete  (24 CPUs, 31.8 GB RAM, 897.7/930.7 GB disk)


In [3]:
model = YOLO('yolov8s.pt')
# model.to(device)

In [4]:
root_dir = 'C:/Users/faree/Documents/School/CSCI4052U_Final_Project/BoardParser'

In [5]:
train_path = os.path.join(root_dir, 'datasets', 'roboflow', 'dataset','train', 'images')
yaml_path = os.path.join(root_dir, 'datasets', 'roboflow', 'dataset', 'data.yaml')
valid_path = os.path.join(root_dir, 'datasets', 'roboflow', 'dataset','val', 'images')

In [6]:
results = model.train(
    data=yaml_path,
    epochs=15,
    batch=32,
    lr0=0.0001,
    lrf=0.1,
    imgsz=640,
    plots=True
)

Ultralytics 8.3.5  Python-3.11.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:/Users/faree/Documents/School/CSCI4052U_Final_Project/BoardParser\datasets\roboflow\dataset\data.yaml, epochs=15, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, s

NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PythonDispatcher].

CPU: registered at C:\Users\circleci\project\torchvision\csrc\ops\cpu\nms_kernel.cpp:112 [kernel]
QuantizedCPU: registered at 

In [31]:
data_dir = '/runs/detect/train4'

img_dir = os.path.join(data_dir, *g")

files = glob.glob(img_dir)

imgs = []

for image in files:
    imgs.append(cv2.imread(image))
    plt.figure(figsize=(10, 10), dpi=200)
    plt.imshow(img)

In [ ]:
out = model.predict(source='image.png', conf=0.5, save=True)